In [1]:
import pandas as pd
import numpy as np
import re
import string
import ast
import os, glob
from sklearn.externals import joblib
import name_extractor
from address_extractor import *
from information_extractor import *
from number_extractor import *
import random
import html2text
import csv
from operator import itemgetter
from collections import defaultdict
from scipy import stats

In [2]:
def clean_dataset_addr(data):
    dataMask = data.addr_index.apply(lambda x: x != [])
    data = data[dataMask].reset_index(drop=True)
    return data

In [3]:
def clean_dataset_inf(data):
    dataMask = data.inf_index.apply(lambda x: x != [])
    data = data[dataMask].reset_index(drop=True)
    return data

In [4]:
def clean_dataset_number(data):
    dataMask = data.numbers_index.apply(lambda x: x != [])
    data = data[dataMask].reset_index(drop=True)
    return data

In [5]:
def clean_dataset(data):
    persons_to_remove = [("Бокаріус","М.С."), ("Порошен","П.О."), ("Шевчен","Т.Г."), 
                         ("Ленін","В.І."), ("Ленин","В.И."), ("Україн","У.П.")]
    dataMask = data.person_indexes.apply(lambda x: x != [])
    data = data[dataMask].reset_index(drop=True)
    dataCleanMask = data.person.apply(lambda x: x not in persons_to_remove)
    data = data[dataCleanMask].reset_index(drop=True)
    data = data[data.person.apply(lambda x: bool(re.search('[А-Яа-яІЇҐЄіїєґ]', x[0])))].reset_index(drop=True)
    return data

In [6]:
def distanses_to_judge(raw_data, df):
    dist_judge_1 = pd.Series([np.nan]*len(raw_data))
    dist_judge_n = pd.Series([np.nan]*len(raw_data))
    raw_data['MIN'] = raw_data['person_indexes'].apply(min)
    raw_data['MAX'] = raw_data['person_indexes'].apply(max)
    isJudgesMask = raw_data.judge.apply(lambda x: len(x) != 0)
    if sum(isJudgesMask) != 0:
        dist_judge_n[isJudgesMask] = raw_data[isJudgesMask].apply(lambda row: [row['MAX']-i for i in row['judge'] if i<=row['MAX']], axis=1)
        dist_judge_1[isJudgesMask] = raw_data[isJudgesMask].apply(lambda row: [row['MIN']-i for i in row['judge'] if i<=row['MIN']], axis=1)
        dist_judge_1[isJudgesMask] = dist_judge_1[isJudgesMask].apply(lambda x: -1 if len(x)==0 else min(x))
        dist_judge_n[isJudgesMask] = dist_judge_n[isJudgesMask].apply(lambda x: -1 if len(x)==0 else min(x))
    df['dist_1_judges'] = dist_judge_1
    df['dist_n_judges'] = dist_judge_n
    return df

def distanses_to_label(raw_data, column, col_to, df):
    dist_label = pd.Series([np.nan]*len(raw_data))
    isLabelMask = raw_data[column].apply(lambda x: len(x) != 0)
    dist_label[isLabelMask] = raw_data[isLabelMask]\
                              .apply(lambda row: [i-j for i in row[col_to] for j in row[column]], axis=1)\
                              .apply(lambda x: -1 if np.all([i<0 for i in x]) else min([i for i in x if i >=0]))
    df['min_dist_'+column] = dist_label
    return df

def distanses_to_label_positive(raw_data, column, col_to, df):
    dist_label = pd.Series([np.nan]*len(raw_data))
    isLabelMask = raw_data[column].apply(lambda x: len(x) != 0)
    dist_label[isLabelMask] = raw_data[isLabelMask]\
                              .apply(lambda row: [i-j for i in row[col_to] for j in row[column]], axis=1)\
                              .apply(lambda x: -1 if np.all([i<=0 for i in x]) else min([i for i in x if i > 0]))
    df['min_dist_positive_'+column] = dist_label
    return df

def distanses_to_label_negative(raw_data, column, col_to, df):
    dist_label = pd.Series([np.nan]*len(raw_data))
    isLabelMask = raw_data[column].apply(lambda x: len(x) != 0)
    dist_label[isLabelMask] = raw_data[isLabelMask]\
                              .apply(lambda row: [i-j for i in row[col_to] for j in row[column]], axis=1)\
                              .apply(lambda x: -1 if np.all([i>=0 for i in x]) else max([i for i in x if i <0]))
    df['max_dist_negative_'+column] = dist_label
    return df

In [7]:
def prepare_dataset_for_model_persons(data, main_labels):
    
    cols_indexes = list(data.select_dtypes(include=['O']).columns)
    cols1 = ['court_type', 'file_name', 'labels', 'pseudo_labels', 'person_indexes','person']
    cols2 = ['judge', 'representative', 'plaintiff', 'defendant', 'defence', 'liquidator']
    cols_indexes = [i for i in cols_indexes if i not in cols1]
    data = clean_dataset(data)
    df = data[['person', 'file_name']]
    df['count'] = data['person_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['first_index'] = data['person_indexes'].apply(lambda x: x[0])
    df['last_index'] = data['length_doc'] - data['person_indexes'].apply(lambda x: x[-1])
    df['length_doc'] = data['length_doc']
    df['first_index %'] = round(df['first_index']/df['length_doc'], 5)
    df['last_index %'] = round(data['person_indexes'].apply(lambda x: x[-1])/data['length_doc'], 5)
    df = distanses_to_judge(data, df)
    for col in cols_indexes:
        df = distanses_to_label(data, col, 'person_indexes', df)
    df['count_of_persons_in_doc'] = df['file_name'].groupby(df['file_name']).transform('count')
    df['person_last_name'] = df.person.apply(lambda x: x[0])
    df['count_with_namesakes'] = df.groupby(['file_name','person_last_name'])['count'].transform('sum') - df['count']
    all_persons_indexes = data.groupby(['file_name'])['person_indexes'].apply(lambda x: sum(x, []))
    all_persons = pd.DataFrame(all_persons_indexes).reset_index(level=0)
    all_persons['all_persons_indexes'] = all_persons['person_indexes']
    all_persons.drop(['person_indexes'], axis=1, inplace=True)
    data = data.merge(all_persons, on='file_name', how='left')
    df = distanses_to_label_positive(data, 'all_persons_indexes', 'person_indexes', df)
    df['std_persons_indexes'] = data[['all_persons_indexes', 'length_doc']].apply(lambda row: np.std([x/row['length_doc'] for x in row['all_persons_indexes']]), axis=1)
    for col in cols2+['quotes']:
        df = distanses_to_label_negative(data, col, 'person_indexes', df)
    data['labels_first_stem'] = data['labels'].apply(lambda x: x[0].split(' ')[0].lower() 
                                                     if len(x)>0 else '').apply(lambda x: name_extractor.UkrainianStemmer(x).stem_word())
    data['pseudo_labels_stem'] = data['pseudo_labels'].apply(lambda x: x[0].lower() 
                                                             if len(x)>0 else '').apply(lambda x: name_extractor.UkrainianStemmer(x).stem_word())
    data['label_&_pseudo'] = data[['labels_first_stem','pseudo_labels_stem']].apply(lambda row: row['labels_first_stem'] if row['labels_first_stem']!=''
                                                                       else row['pseudo_labels_stem'], axis=1)
    data['label_&_pseudo'] = data['label_&_pseudo'].apply(lambda x: 'судд' if 'судд' in x else x)
    data['label_&_pseudo'] = data['label_&_pseudo'].apply(lambda x: 'фоп' if 'підприємец' in x else x)
    data['label_&_pseudo'] = data['label_&_pseudo'].apply(lambda x: 'доньк' if 'дочк' in x else x)
    data['label_&_pseudo'] = data['label_&_pseudo'].apply(lambda x: 'захисник' if 'захисник' in x else x)
    df['label_&_pseudo'] = data['label_&_pseudo'].apply(lambda x: -1 if x not in main_labels.values() 
                                                    else [k for k, v in main_labels.items() if v == x][0])
    for col in cols2:
        df['count_'+col] = data[col].apply(len) 
    df['criminal_court'] = data.court_type.apply(lambda x: any(w.lower().startswith('кримінальн') for w in x.split(' ')))*1
    df['civil_court'] = data.court_type.apply(lambda x: any(w.lower().startswith('цивільн') for w in x.split(' ')))*1
    df['administr_court'] = data.court_type.apply(lambda x: any(w.lower().startswith('адміністр') for w in x.split(' ')))*1
    df['hosp_court'] = data.court_type.apply(lambda x: any(w.lower().startswith('господар') for w in x.split(' ')))*1
    df['apel_court'] = data.court_type.apply(lambda x: any(w.lower().startswith('апеляційн') for w in x.split(' ')))*1
    df['spec_court'] = data.court_type.apply(lambda x: any(w.lower().startswith('спеціалізован') for w in x.split(' ')))*1
    
    return df
    

In [8]:
def prepare_dataset_for_model_address(data):
    cols_to_list = list(data.select_dtypes(include=['O']).columns)
    data = clean_dataset_addr(data)
    
    df = data[['addresses', 'file_name']]
    df['addr_count'] = data['addr_count']
    df['count_of_adresses_in_doc'] = data['file_name'].groupby(data['file_name']).transform('count')
    df['count'] = data['addr_index'].apply(lambda x: len(x) if len(x)>0 else np.nan) 
    
    cols_to_list.remove('file_name')
    cols_to_list.remove('addresses')
    cols_to_list.remove('addr_index')
    for col in cols_to_list:
        df = distanses_to_label(data, col, 'addr_index', df)
        df = distanses_to_label_negative(data, col, 'addr_index', df)
        
    all_addresses_indexes = data.groupby(['file_name'])['addr_index'].apply(lambda x: sum(x, []))
    all_addresses = pd.DataFrame(all_addresses_indexes).reset_index(level=0)
    all_addresses['all_addresses_indexes'] = all_addresses['addr_index']
    all_addresses.drop(['addr_index'], axis=1, inplace=True) 
    data = data.merge(all_addresses, on='file_name', how='left')
    

    df['addr_len_digit'] = data['addr_len_digit']
    df['addr_is_appartment1'] = data['addr_is_appartment1']
    df['addr_is_appartment2'] = data['addr_is_appartment2']
    df['addr_is_street'] = data['addr_is_street']
    
    df['count_residence_indexes'] = data['residence_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan) 
    df['count_inherited_indexes'] = data['inherited_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan) 
    df['count_registered_indexes'] = data['registered_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan) 
    df['count_house_indexes'] = data['house_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan) 
    df['count_legal_address_indexes'] = data['legal_address_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_property_indexes'] = data['property_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_barn_indexes'] = data['barn_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_land_plot_indexes'] = data['land_plot_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_flat_indexes'] = data['flat_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_pull_indexes'] = data['pull_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_auto_indexes'] = data['auto_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    df['count_search_indexes'] = data['search_indexes'].apply(lambda x: len(x) if len(x)>0 else np.nan)
    
    return df

In [9]:
def prepare_dataset_for_model_information(data):
    cols_to_list = list(data.select_dtypes(include=['O']).columns)
    data = clean_dataset_inf(data)
    df = data[['file_name','informations']] 
    return df

In [10]:
def prepare_dataset_for_model_number(data):
    cols_to_list = list(data.select_dtypes(include=['O']).columns)
    data = clean_dataset_number(data)
    df = data[['file_name','numbers']] 
    return df

In [11]:
def model_eval_persons(df, model_lgb, model_lr):
    X = df.drop(['person', 'person_last_name', 'file_name'], axis=1)
    X.fillna(-10, inplace=True)
    y_pred_lgb = model_lgb.predict(X.values)
    y_pred_lr = model_lr.predict_proba(X.values)[:,1]
    y_predictions = 0.98*y_pred_lgb + 0.02*y_pred_lr
    df['to_depers'] = (y_predictions > 0.5)
    data_to_depers = df[['file_name','person']][df.to_depers]
    return data_to_depers

In [12]:
def model_eval_address(df, model_lgb, model_lr):
    X = df.drop(['addresses', 'file_name'], axis = 1)
    X.fillna(-10, inplace=True)
    y_pred_lgb = model_lgb.predict(X.values)
    y_pred_lr = model_lr.predict_proba(X.values)[:,1]
    y_predictions = 0.98*y_pred_lgb + 0.02*y_pred_lr
    df['probability'] = y_predictions
    df['to_depers'] = (y_predictions > 0.5)
    data_to_depers = df[['file_name','addresses', 'probability']][df.to_depers]
    return data_to_depers

In [13]:
def model_eval_information(df):
    X = df.drop(['informations', 'file_name'], axis = 1)
    X.fillna(-10, inplace=True)
    data_to_depers = df[['file_name','informations']]
    return data_to_depers

In [14]:
def model_eval_number(df):
    X = df.drop(['numbers', 'file_name'], axis = 1)
    X.fillna(-10, inplace=True)
    data_to_depers = df[['file_name','numbers']]
    return data_to_depers

In [15]:
def strip(string):
    return re.sub('[\'`’-]', '', string)

In [16]:
def depersonalization_person(file_name, raw_data, to_depers):
    persons = {}
    for person in enumerate(to_depers):
        persons[person[1]] = 'ОСОБА_' + str(person[0]+1)
    txt = re.split('([^A-Za-zА-Яа-яІЇЄҐіїєґ0-9-\'`’])', raw_data)
    reg = re.compile('[A-Za-zА-Яа-яІЇЄҐіїєґ0-9-\'`’]')
    states = [(bool(reg.match(i)) & (i != 'nbsp')) for i in txt]
    indexes = np.where(states)[0]
    txt1 = [strip(i) for i in txt]
    lst = list(zip(indexes,itemgetter(*indexes)(txt1)))
    ind_to_person = defaultdict(list)
    ind_to_drop = []
    for person in persons:
        for i in range(2,len(lst)-2):
            betw_names1 = range(lst[i-1][0]+1, lst[i][0])
            betw_names2 = range(lst[i][0]+1, lst[i+1][0])
            try:
                if (lst[i][1].startswith(person[0]) | lst[i][1].startswith(person[0].upper())) &\
                    (lst[i-2][1].startswith(person[1][0]) & lst[i-1][1].startswith(person[1][2]) &\
                    all([(txt[j]!=',') for j in betw_names1])):
                    ind_to_person[person].append(lst[i][0])
                    if (len(lst[i-1][1]) == 1) & (txt[lst[i-1][0]+1] =='.'):
                        ind_to_drop.extend(range(lst[i-2][0]-1, lst[i-1][0]+2))
                    else:
                        ind_to_drop.extend(range(lst[i-2][0]-1, lst[i-1][0]+1))
                elif (lst[i][1].startswith(person[0]) | lst[i][1].startswith(person[0].upper())) &\
                    (lst[i+1][1].startswith(person[1][0]) & lst[i+2][1].startswith(person[1][2]) &\
                    all([(txt[j]!=',') for j in betw_names2])):
                    ind_to_person[person].append(lst[i][0])
                    if (len(lst[i+2][1]) == 1) & (txt[lst[i+2][0]+1] == '.'):
                        ind_to_drop.extend(range(lst[i+1][0]-1, lst[i+2][0]+2))
                    else:
                        ind_to_drop.extend(range(lst[i+1][0]-1, lst[i+2][0]+1))
                elif (lst[i][1].startswith(person[0]) | lst[i][1].startswith(person[0].upper())) &\
                    (lst[i-1][1].startswith(person[1][0]) & \
                    all([(txt[j]!=',') for j in betw_names1]) &((len(lst[i-1][1]) <=2) | (lst[i-1][1][-1].islower()))):
                    ind_to_person[person].append(lst[i][0])
                    if (len(lst[i-1][1]) == 1) & (txt[lst[i-1][0]+1] =='.'):
                        ind_to_drop.extend(range(lst[i-1][0], lst[i-1][0]+2))
                    else:
                        ind_to_drop.append(lst[i-1][0])   
                elif (lst[i][1].startswith(person[0]) | lst[i][1].startswith(person[0].upper())) &\
                    (lst[i+1][1].startswith(person[1][0]) & all([(txt[j]!=',') for j in betw_names2]) & ((len(lst[i+1][1]) <=2) | (lst[i+1][1][-1].islower()))):
                    ind_to_person[person].append(lst[i][0])
                    if (len(lst[i+1][1]) == 1) & (txt[lst[i+1][0]+1] =='.'):
                        ind_to_drop.extend(range(lst[i+1][0], lst[i+1][0]+2))
                    else:
                        ind_to_drop.append(lst[i+1][0])
            except IndexError:
                pass
    S = np.array(txt)
    dct_depers = {k : (persons[k], ind_to_person[k]) for k in persons.keys() & ind_to_person.keys()}
    for i in dct_depers:
        S[dct_depers[i][1]] = dct_depers[i][0]
    S[ind_to_drop] = ''
    new_text = ''.join(S)
    
    return new_text

In [17]:
def depersonalization_addresses(file_name, raw_data, to_depers):
    addresses = {}
    for addr in enumerate(to_depers.keys()):
        addresses[addr[1]] = 'АДРЕСА_' + str(addr[0]+1)
    dct_depers = {k : (addresses[k], to_depers[k]) for k in addresses.keys() & to_depers.keys()}
    S = np.array(AddressSearch.clean_text_to_list(raw_data)[1])
    for i in dct_depers:
        for v in dct_depers[i][1]:
            S[v['start']] = dct_depers[i][0]
            S[v['start']+1:v['stop']+1] = ''
    new_text = ''.join(S)
    return new_text

In [18]:
def depersonalization_informations(file_name, raw_data, to_depers):
    informations = {}
    for inf in enumerate(to_depers.keys()):
        informations[inf[1]] = 'ІНФОРМАЦІЯ_' + str(inf[0]+1)
    dct_depers = {k : (informations[k], to_depers[k]) for k in informations.keys() & to_depers.keys()}
    S = np.array(InformationSearch.clean_text_to_list(raw_data)[1])
    for i in dct_depers:
        for v in dct_depers[i][1]:
            S[v['start']] = dct_depers[i][0]
            S[v['start']+1:v['stop']+1] = ''
    new_text = ''.join(S)
    return new_text

In [19]:
def depersonalization_numbers(file_name, raw_data, to_depers):
    numbers = {}
    for number in enumerate(to_depers.keys()):
        numbers[number[1]] = 'НОМЕР_' + str(number[0]+1)
    dct_depers = {k : (numbers[k], to_depers[k]) for k in numbers.keys() & to_depers.keys()}
    S = np.array(NumberSearch.clean_text_to_list(raw_data)[1])
    for i in dct_depers:
        for v in dct_depers[i][1]:
            S[v['start']] = dct_depers[i][0]
            S[v['start']+1:v['stop']+1] = ''
    new_text = ''.join(S)
    return new_text

In [20]:
%%time
if __name__ == '__main__':
    dir_before_depers = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'data_original/*.html')
    dir_after_depers = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'data_imp/')
    datasets = []
    datasets_ad = []
    datasets_inf = []
    datasets_number = []
    
    # sample of 6 docs
#     random.seed(2)
#     files = random.sample(glob.glob(dir_before_depers), 6)
    
    files = glob.glob(dir_before_depers)
    count = 0
    with open('main_labels.csv') as csv_file:
        reader = csv.reader(csv_file)
        main_labels = dict(reader)
    model_lgb = joblib.load('lgbm.pkl')
    model_lr = joblib.load('lr.pkl')
    model_lgb_addr = joblib.load('lgbm_address.pkl')
    model_lr_addr = joblib.load('lr_address.pkl')
    
    for x in files:
        with open(x, 'r') as fdata:
            raw_text = fdata.read()
        text = html2text.html2text(raw_text)
        file = os.path.splitext(os.path.basename(x))[0]
        
        count+=1
        print(count, file)
        
        number = NumberSearch.find_unique_numbers(NumberSearch(raw_text).numbers_search())
        if number:
            data_number = FeaturesForNumber(file, raw_text).make_dataframe()  
            df_number = prepare_dataset_for_model_number(clean_dataset_number(data_number))
            to_dep_set_number =  model_eval_number(df_number)
            
            to_depers_number = {i:number[i] for i in number if i in to_dep_set_number['numbers'].values}
            if  to_depers_number:
                raw_text = depersonalization_numbers(file, raw_text, to_depers_number)
                datasets_number.append(to_dep_set_number)
        
        information = InformationSearch.find_unique_informations(InformationSearch(raw_text).information_search())
        if information:
            data_inf = FeaturesForInformation(file, raw_text).make_dataframe()  
            df_inf = prepare_dataset_for_model_information(clean_dataset_inf(data_inf))
            to_dep_set_inf =  model_eval_information(df_inf)
            
            to_depers_inf = {i:information[i] for i in information if i in to_dep_set_inf['informations'].values}
            if  to_depers_inf:
                raw_text = depersonalization_informations(file, raw_text, to_depers_inf)
                datasets_inf.append(to_dep_set_inf)
               
        addresses = AddressSearch.find_unique_address(AddressSearch(raw_text).adress_search())
        if addresses:
            data_ad = FeaturesForAddresses(file, raw_text).make_dataframe()
            df_ad = prepare_dataset_for_model_address(clean_dataset_addr(data_ad))
            to_dep_set = model_eval_address(df_ad, model_lgb_addr, model_lr_addr)
            to_depers_ad = {i:addresses[i] for i in addresses if i in to_dep_set['addresses'].values}
            if to_depers_ad:
                raw_text = depersonalization_addresses(file, raw_text, to_depers_ad)
                datasets_ad.append(to_dep_set)
          
        persons = name_extractor.NameSearch(text).name_searh()
        if persons:
            data = name_extractor.FeaturesForNames(file, text, persons).make_dataframe()
            df = prepare_dataset_for_model_persons(clean_dataset(data), main_labels)
            to_depers = model_eval_persons(df, model_lgb, model_lr)
            if not to_depers.empty:
                new_text = depersonalization_person(file, raw_text, to_depers['person'].values)
                datasets.append(to_depers)
            else:
                new_text = raw_text
        else:
            new_text = raw_text
        with open(dir_after_depers+str(file)+'_imp.html', 'w') as wfile:
            wfile.write(new_text)
            
               
    if datasets:
        df_to_depers = pd.concat(datasets, axis=0, ignore_index=True)
        df_to_depers.to_csv('test_persons.csv', index=False)
    if datasets_ad:
        df_to_depers_ad = pd.concat(datasets_ad, axis=0, ignore_index=True)
        df_to_depers_ad.to_csv('test_addresses.csv', index=False)
    if datasets_inf:
        to_depers_inf = pd.concat(datasets_inf, axis=0, ignore_index=True)
        to_depers_inf.to_csv('test_informations.csv', index=False)
    if datasets_number:
        to_depers_number = pd.concat(datasets_number, axis=0, ignore_index=True)
        to_depers_number.to_csv('test_numbers.csv', index=False)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


1 79991364


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

ERROR in NER.find_unique_persons:  'NoneType' object is not subscriptable
ERROR in NER.find_unique_persons:  'NoneType' object is not subscriptable
ERROR in NER.find_unique_persons:  'NoneType' object is not subscriptable


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

...

49 79992473
615
615


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

50 79992474


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

Wall time: 19min 58s


In [21]:
pd.read_csv('test_numbers.csv')

,file_name,numbers
0,79991516,{'2173304013'}
1,79991516,{'2189021043'}
2,79991516,{'1016218725'}
3,79991523,{'І-КЕ'}
4,79991523,"{'І-КЕ', '184271'}"
5,79991523,{'2447817670'}
6,79991697,{'АВ'}
7,79991697,"{'ВО', 'АВ', '9583'}"
8,79991697,"{'6810', 'ВІ', 'АВ'}"
9,79991697,"{'9588', 'АВ', 'ВО'}"
